<a href="https://colab.research.google.com/github/darshan12x/projects/blob/mainbranch/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import tweepy
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import numpy as np 
import matplotlib.pyplot as plt 
import string
import nltk
import warnings
import seaborn as sns
warnings.filterwarnings("ignore", category=DeprecationWarning)
nltk.download('vader_lexicon')

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [53]:
#My Twitter API Authentication Variables
consumer_key = 'VTgswlznw8Dzbj42HaL98NOAs'
consumer_secret = 'QDR7gDKuUzc6T2I6R96FZ449GdWXNrELrcAdK8S1GZEutd0DTp'
access_token = '957181535483908096-GdXFwmb1AFa0gmkhNwsDChpIOw2OY8r'
access_token_secret = '1VouYOsjd9TVK0cMksF4yHqZU3UOVTzg8KSxH2VtnAj9c'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
tweets = api.search('Irfan Khan', count=200)

data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

display(data.head(10))

,Tweets
0,RT @fayedsouza: What was your favourite Film o...
1,One of my favourite actor. Rest in Peace Irfan...
2,RT @sambitswaraj: Such a sad sad news of Actor...
3,"An artist who was blessed by God, like Bali, t..."
4,RT @dhruv_rathee: Heartbreaking to hear about ...
5,RT @ActorMadhavan: Actor Irrfan Khan Dies In M...
6,RT @sambitswaraj: Such a sad sad news of Actor...
7,RT @CricNerd_vKash: Never thought we'd be talk...
8,RT @katrinakaif_4: @SrBachchan irfan khan is n...
9,RT @dhruv_rathee: Heartbreaking to hear about ...


In [54]:
# tidy up tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

# remove twitter handles (@user)
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['Tweets'], "@[\w]*")

# remove special characters, numbers, punctuations
data['tidy_tweet'] = data['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

# removing short words
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

display(data.head(10))

,Tweets,tidy_tweet
0,RT @fayedsouza: What was your favourite Film o...,What was your favourite Film character played ...
1,One of my favourite actor. Rest in Peace Irfan...,One favourite actor Rest Peace Irfan Khan
2,RT @sambitswaraj: Such a sad sad news of Actor...,Such sad sad news Actor Irfan Khan passing awa...
3,"An artist who was blessed by God, like Bali, t...",artist who was blessed God like Bali that you ...
4,RT @dhruv_rathee: Heartbreaking to hear about ...,Heartbreaking hear about Irfan Khan death was ...
5,RT @ActorMadhavan: Actor Irrfan Khan Dies In M...,Actor Irrfan Khan Dies Mumbai This such traged...
6,RT @sambitswaraj: Such a sad sad news of Actor...,Such sad sad news Actor Irfan Khan passing awa...
7,RT @CricNerd_vKash: Never thought we'd be talk...,Never thought talking about Irfan Khan past te...
8,RT @katrinakaif_4: @SrBachchan irfan khan is n...,irfan khan more #RIP #IrfanKhan #RIPirfan http...
9,RT @dhruv_rathee: Heartbreaking to hear about ...,Heartbreaking hear about Irfan Khan death was ...


In [55]:
sid = SentimentIntensityAnalyzer()
listy = []

for index, row in data.iterrows():
  ss = sid.polarity_scores(row["tidy_tweet"])
  listy.append(ss)
  
se = pd.Series(listy)
data['polarity'] = se.values

display(data.head(100))

,Tweets,tidy_tweet,polarity
0,RT @fayedsouza: What was your favourite Film o...,What was your favourite Film character played ...,"{'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'comp..."
1,One of my favourite actor. Rest in Peace Irfan...,One favourite actor Rest Peace Irfan Khan,"{'neg': 0.0, 'neu': 0.632, 'pos': 0.368, 'comp..."
2,RT @sambitswaraj: Such a sad sad news of Actor...,Such sad sad news Actor Irfan Khan passing awa...,"{'neg': 0.221, 'neu': 0.536, 'pos': 0.243, 'co..."
3,"An artist who was blessed by God, like Bali, t...",artist who was blessed God like Bali that you ...,"{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'comp..."
4,RT @dhruv_rathee: Heartbreaking to hear about ...,Heartbreaking hear about Irfan Khan death was ...,"{'neg': 0.265, 'neu': 0.462, 'pos': 0.273, 'co..."
...,...,...,...
95,not irfan khan 😭 https://t.co/b9gMJTMgjW,not irfan khan https gMJTMgjW,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
96,RT @VishalVerma_9: This feels there right in t...,This feels there right the middle Gone too soo...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
97,यही हमारे संस्कार है....\nहम इंसान को इंसान सम...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
98,Everyone in the Industry is in for Money while...,Everyone the Industry for Money while Irfan kh...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [56]:
#  tokenize all the cleaned tweets in our dataset
tokenized_tweet = data['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

0    [What, was, your, favourite, Film, character, ...
1    [One, favourite, actor, Rest, Peace, Irfan, Khan]
2    [Such, sad, sad, news, Actor, Irfan, Khan, pas...
3    [artist, who, was, blessed, God, like, Bali, t...
4    [Heartbreaking, hear, about, Irfan, Khan, deat...
Name: tidy_tweet, dtype: object